In [ ]:
def get_winter_ranges(start_year, end_year):
    ranges = []
    for y in range(start_year, end_year):
        ranges.append((
            f"{y}-10-01",
            f"{y+1}-01-31"
        ))
    return ranges

print(get_winter_ranges(2022,2026))

[('2022-10-01', '2023-01-31'), ('2023-10-01', '2024-01-31'), ('2024-10-01', '2025-01-31'), ('2025-10-01', '2026-01-31')]


In [ ]:
import requests
import pandas as pd

def fetch_openmeteo(lat, lon, city, start_date, end_date, pollutants):
    print(f"Fetching AQ data: {city} | {start_date} → {end_date}")

    params = ",".join(pollutants)

    url = (
        "https://air-quality-api.open-meteo.com/v1/air-quality"
        f"?latitude={lat}&longitude={lon}"
        f"&hourly={params}"
        f"&start_date={start_date}&end_date={end_date}"
    )

    r = requests.get(url, timeout=30)
    r.raise_for_status()
    data = r.json()

    if "hourly" not in data or "time" not in data["hourly"]:
        raise ValueError(f"No AQ data returned for {city}")

    hourly = data["hourly"]

    df = pd.DataFrame({
        "datetime": pd.to_datetime(hourly["time"]),
    })

    for p in pollutants:
        col = p.upper().replace(".", "_")
        df[col] = hourly.get(p, [])

    df["CITY"] = city
    df.sort_values("datetime", inplace=True)
    df.reset_index(drop=True, inplace=True)
    return df



In [2]:
import pandas as pd

def fetch_all_winters_openmeteo(lat, lon, city, winter_ranges, pollutants):
    frames = []

    for start_date, end_date in winter_ranges:
        df = fetch_openmeteo(
            lat=lat,
            lon=lon,
            city=city,
            start_date=start_date,
            end_date=end_date,
            pollutants=pollutants
        )
        frames.append(df)

    full_df = pd.concat(frames, ignore_index=True)

    # Remove duplicates caused by overlapping API edges
    full_df.drop_duplicates(subset=["datetime", "CITY"], inplace=True)

    # Enforce hourly continuity
    full_df.sort_values("datetime", inplace=True)
    full_df.reset_index(drop=True, inplace=True)

    return full_df


In [4]:
pollutants = [
    "pm10",
    "pm2_5",
    "nitrogen_dioxide",
    "sulphur_dioxide",
    "ozone",
    "carbon_monoxide"
]


In [8]:
winter_ranges = [
    ("2022-10-01", "2023-01-31"),
    ("2023-10-01", "2024-01-31"),
    ("2024-10-01", "2025-01-31"),
]

delhi_lat, delhi_lon = 28.6139, 77.2090

aqi_df = fetch_all_winters_openmeteo(
    lat=delhi_lat,
    lon=delhi_lon,
    city="Delhi",
    winter_ranges=winter_ranges,
    pollutants=pollutants
)

aqi_df.head()


Fetching AQ data: Delhi | 2022-10-01 → 2023-01-31
Fetching AQ data: Delhi | 2023-10-01 → 2024-01-31
Fetching AQ data: Delhi | 2024-10-01 → 2025-01-31


,datetime,PM10,PM2_5,NITROGEN_DIOXIDE,SULPHUR_DIOXIDE,OZONE,CARBON_MONOXIDE,CITY
0,2022-10-01 00:00:00,193.2,127.6,88.3,71.8,12.0,1705.0,Delhi
1,2022-10-01 01:00:00,177.0,118.0,88.6,63.5,16.0,1684.0,Delhi
2,2022-10-01 02:00:00,166.2,111.5,88.7,51.8,22.0,1649.0,Delhi
3,2022-10-01 03:00:00,151.8,100.8,81.7,39.8,41.0,1499.0,Delhi
4,2022-10-01 04:00:00,120.6,78.5,65.3,37.9,73.0,1228.0,Delhi


In [10]:
expected_hours = (
    (aqi_df["datetime"].max() - aqi_df["datetime"].min()).total_seconds() / 3600
) + 1

print("Expected hours:", int(expected_hours))
print("Actual rows   :", len(aqi_df))
aqi_df.to_csv('output.csv')

Expected hours: 20496
Actual rows   : 8856


In [13]:
NASA_PARAMS = ",".join([
    "T2M",                  # Temperature at 2m
    "RH2M",                 # Relative humidity
    "WS2M",                 # Wind speed
    "WD2M",                 # Wind direction
    "PRECTOTCORR",          # Precipitation
    "ALLSKY_SFC_SW_DWN"     # Solar irradiance
])


In [12]:
winter_ranges = [
    ("2022-10-01", "2023-01-31"),
    ("2023-10-01", "2024-01-31"),
    ("2024-10-01", "2025-01-31")
]


In [8]:
from datetime import datetime, timedelta

def split_into_chunks(start_date, end_date, chunk_days=30):
    start = datetime.strptime(start_date, "%Y-%m-%d")
    end = datetime.strptime(end_date, "%Y-%m-%d")

    chunks = []
    current = start

    while current <= end:
        chunk_end = min(current + timedelta(days=chunk_days), end)
        chunks.append((
            current.strftime("%Y%m%d"),
            chunk_end.strftime("%Y%m%d")
        ))
        current = chunk_end + timedelta(days=1)

    return chunks


In [9]:
import requests
import pandas as pd

def fetch_nasa_power_chunk(lat, lon, start, end, params,
                           community="RE",
                           time_standard="UTC"):
    url = (
        "https://power.larc.nasa.gov/api/temporal/hourly/point"
        f"?start={start}&end={end}"
        f"&latitude={lat}&longitude={lon}"
        f"&parameters={params}"
        f"&community={community}"
        f"&time-standard={time_standard}"
        f"&format=JSON"
    )

    print(f"NASA → {start} to {end}")
    r = requests.get(url, timeout=30)
    r.raise_for_status()
    data = r.json()

    try:
        params_dict = data["properties"]["parameter"]
    except KeyError:
        raise ValueError(f"Invalid NASA response: {data}")

    df = pd.DataFrame(params_dict)
    df.index.name = "datetime"
    df.reset_index(inplace=True)

    df["datetime"] = pd.to_datetime(df["datetime"], format="%Y%m%d%H")
    df.sort_values("datetime", inplace=True)
    df.reset_index(drop=True, inplace=True)

    return df


In [10]:
def fetch_all_winters_nasa(lat, lon, winter_ranges, params):
    frames = []

    for start_date, end_date in winter_ranges:
        chunks = split_into_chunks(start_date, end_date)

        for start, end in chunks:
            df_chunk = fetch_nasa_power_chunk(
                lat=lat,
                lon=lon,
                start=start,
                end=end,
                params=params
            )
            frames.append(df_chunk)

    df = pd.concat(frames, ignore_index=True)

    # De-duplicate overlapping edges
    df.drop_duplicates(subset="datetime", inplace=True)

    df.sort_values("datetime", inplace=True)
    df.reset_index(drop=True, inplace=True)

    return df


In [16]:
delhi_lat, delhi_lon = 28.6139, 77.2090

nasa_df = fetch_all_winters_nasa(
    lat=delhi_lat,
    lon=delhi_lon,
    winter_ranges=winter_ranges,
    params=NASA_PARAMS
)

nasa_df.head()


NASA → 20221001 to 20221031
NASA → 20221101 to 20221201
NASA → 20221202 to 20230101
NASA → 20230102 to 20230131
NASA → 20231001 to 20231031
NASA → 20231101 to 20231201
NASA → 20231202 to 20240101
NASA → 20240102 to 20240131
NASA → 20241001 to 20241031
NASA → 20241101 to 20241201
NASA → 20241202 to 20250101
NASA → 20250102 to 20250131


,datetime,T2M,RH2M,WS2M,WD2M,PRECTOTCORR,ALLSKY_SFC_SW_DWN
0,2022-10-01 00:00:00,23.16,87.95,0.39,295.9,0.00,0.00
1,2022-10-01 01:00:00,23.17,91.82,0.56,264.9,0.00,87.18
2,2022-10-01 02:00:00,24.52,86.87,1.21,259.5,0.00,266.83
3,2022-10-01 03:00:00,26.61,80.77,1.28,266.4,0.00,475.20
4,2022-10-01 04:00:00,29.57,65.00,1.57,286.7,0.01,645.40


In [19]:
nasa_df.shape

(8856, 7)

In [18]:
nasa_df.to_csv('output1.csv')

In [21]:
# Ensure both are sorted
aqi_df = aqi_df.sort_values("datetime").reset_index(drop=True)
nasa_df = nasa_df.sort_values("datetime").reset_index(drop=True)

# Check exact timestamp equality
mismatch = (aqi_df["datetime"] != nasa_df["datetime"]).sum()
print("Mismatched timestamps:", mismatch)



Mismatched timestamps: 0


In [22]:
aqi_df["datetime"] = pd.to_datetime(aqi_df["datetime"], utc=True)
nasa_df["datetime"] = pd.to_datetime(nasa_df["datetime"], utc=True)


In [23]:
aqi_core = aqi_df.drop(columns=["CITY"])


In [24]:
merged_df = pd.merge(
    aqi_core,
    nasa_df,
    on="datetime",
    how="inner"
)


In [25]:
merged_df.to_csv('output2.csv')

In [ ]:
merged_df.shape
merged_df.columns

Index(['datetime', 'PM10', 'PM2_5', 'NITROGEN_DIOXIDE', 'SULPHUR_DIOXIDE',
       'OZONE', 'CARBON_MONOXIDE', 'T2M', 'RH2M', 'WS2M', 'WD2M',
       'PRECTOTCORR', 'ALLSKY_SFC_SW_DWN'],
      dtype='object')

In [11]:
winter_ranges = [
    ("2025-10-01", "2025-12-25"),
]

delhi_lat, delhi_lon = 28.6139, 77.2090

aqi_df = fetch_all_winters_openmeteo(
    lat=delhi_lat,
    lon=delhi_lon,
    city="Delhi",
    winter_ranges=winter_ranges,
    pollutants=pollutants
)

aqi_df.head()

Fetching AQ data: Delhi | 2025-10-01 → 2025-12-25


,datetime,PM10,PM2_5,NITROGEN_DIOXIDE,SULPHUR_DIOXIDE,OZONE,CARBON_MONOXIDE,CITY
0,2025-10-01 00:00:00,51.1,33.4,27.4,19.1,44.0,518.0,Delhi
1,2025-10-01 01:00:00,47.6,35.3,28.9,22.6,47.0,606.0,Delhi
2,2025-10-01 02:00:00,43.9,36.4,30.6,27.7,53.0,740.0,Delhi
3,2025-10-01 03:00:00,38.6,34.5,30.5,32.8,65.0,807.0,Delhi
4,2025-10-01 04:00:00,44.1,33.4,26.7,38.3,87.0,734.0,Delhi


In [14]:
delhi_lat, delhi_lon = 28.6139, 77.2090

winter_ranges = [
    ("2025-10-01", "2025-12-25"),
]


nasa_df = fetch_all_winters_nasa(
    lat=delhi_lat,
    lon=delhi_lon,
    winter_ranges=winter_ranges,
    params=NASA_PARAMS
)

nasa_df.head()


NASA → 20251001 to 20251031
NASA → 20251101 to 20251201
NASA → 20251202 to 20251225


,datetime,T2M,RH2M,WS2M,WD2M,PRECTOTCORR,ALLSKY_SFC_SW_DWN
0,2025-10-01 00:00:00,23.67,94.50,0.26,321.3,0.03,-999.0
1,2025-10-01 01:00:00,23.83,95.90,0.60,327.9,0.08,-999.0
2,2025-10-01 02:00:00,25.01,89.54,1.28,325.8,0.10,-999.0
3,2025-10-01 03:00:00,26.75,78.29,1.37,324.2,0.00,-999.0
4,2025-10-01 04:00:00,28.55,67.67,0.84,336.8,0.00,-999.0


In [15]:
# Ensure both are sorted
aqi_df = aqi_df.sort_values("datetime").reset_index(drop=True)
nasa_df = nasa_df.sort_values("datetime").reset_index(drop=True)

# Check exact timestamp equality
mismatch = (aqi_df["datetime"] != nasa_df["datetime"]).sum()
print("Mismatched timestamps:", mismatch)


Mismatched timestamps: 0


In [16]:
aqi_df["datetime"] = pd.to_datetime(aqi_df["datetime"], utc=True)
nasa_df["datetime"] = pd.to_datetime(nasa_df["datetime"], utc=True)


In [17]:
aqi_core = aqi_df.drop(columns=["CITY"])


In [18]:
merged_df = pd.merge(
    aqi_core,
    nasa_df,
    on="datetime",
    how="inner"
)


In [19]:
merged_df.to_csv('test.csv')